# 先用megent先拟合误差，再预测实际带隙（利用之前的108个数据）

# 再预测所有的已经DFT优化完成的结构的实际带隙，并存储成json文件，再用数据看是否csm八面体扭曲和带隙存在一定关系

In [1]:
import numpy as np
from pymatgen.core import Structure
from megnet.models import MEGNetModel
from megnet.data.crystal import CrystalGraph
import json

2023-12-12 22:03:15.876665: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-12 22:03:15.923668: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-12 22:03:15.923706: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-12 22:03:15.924873: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-12 22:03:15.932091: I tensorflow/core/platform/cpu_feature_guar

In [2]:
with open("/root/home/jupyter/DFT/bandgap/bandgap_json/DFT_bandgapTrain1.json", "r") as f:
    data_train = json.load(f)

In [3]:
structures = []
gaps = []

for item in data_train:
    structure = Structure.from_dict(item['structure'])
    gap = item['real_gap'] - item['bandgap']
    structures.append(structure)
    gaps.append(gap)

In [4]:
with open("/root/home/jupyter/DFT/bandgap/bandgap_json/DFT_bandgap3Test.json", "r") as f:
    data_test = json.load(f)

In [5]:
structures_test = []
DFT_bandgaps = []

for item in data_test:
    structure = Structure.from_dict(item['structure'])
    DFT_bandgap = item['bandgap']
    structures_test.append(structure)
    DFT_bandgaps.append(DFT_bandgap)

In [6]:
DFT_bandgaps

[{'direct': True, 'energy': 1.6165000000000003, 'transition': 'Z-Z'},
 {'direct': True, 'energy': 1.6435, 'transition': 'Z-Z'},
 {'direct': True, 'energy': 1.9423, 'transition': 'Z-Z'},
 {'direct': True, 'energy': 1.7110999999999998, 'transition': 'Z-Z'},
 {'direct': True, 'energy': 1.5245000000000002, 'transition': 'Z-Z'}]

In [7]:
DFT_bandgaps_test = []
for i in DFT_bandgaps:
    DFT_bandgaps_test.append(i['energy'])

In [8]:
with open("/root/home/jupyter/DFT/bandgap/bandgap_json/DFT_bandgap2Test.json", "r") as f:
    data_test2 = json.load(f)

In [10]:
data_test2[0]

{'structure': {'@module': 'pymatgen.core.structure',
  '@class': 'Structure',
  'charge': 0,
  'lattice': {'matrix': [[12.787734833883398,
     4.6395439e-09,
     0.0001114544232935],
    [-3.044480052e-07, 13.01296343422229, 3.687073349e-07],
    [2.45370005896e-05, -2.9869465e-09, 6.518215051538841]],
   'pbc': [True, True, True],
   'a': 12.7877348343691,
   'b': 13.012963434222298,
   'c': 6.518215051585025,
   'alpha': 89.99999840285068,
   'beta': 89.99928494279463,
   'gamma': 90.00000131967583,
   'volume': 1084.6722174643057},
  'sites': [{'species': [{'element': 'Cs', 'occu': 1}],
    'abc': [0.2353852677491624, 0.2499999999996913, 0.5000228908551582],
    'xyz': [3.010056580728901, 3.253240858150094, 3.259283060192158],
    'label': 'Cs',
    'properties': {}},
   {'species': [{'element': 'H', 'occu': 1}],
    'abc': [0.3483412571701052, 0.7500000000005033, 0.499995300304],
    'xyz': [4.454507668441863, 9.759722575795951, 3.2591159928446443],
    'label': 'H',
    'propert

In [11]:
structure_test2 = Structure.from_dict(data_test2[0]['structure'])
DFT_bandgap = data_test2[0]['bandgap']['energy']


In [13]:
structure_test2.formula #有一个测试集计算错误,只提取第一个

'Cs1 H15 Pb4 C3 I12 N6'

In [14]:
DFT_bandgap

1.5330999999999997

In [15]:
structures_test.append(structure_test2)

In [16]:
DFT_bandgaps_test.append(DFT_bandgap)

In [17]:
print(len(structures_test))
print(len(DFT_bandgaps_test))


6
6


In [18]:
real_gaps_test = [ 1.75, 2.14, 2.28, 1.83, 1.61,1.57] 

In [19]:
for i in structures_test:
    print(i.formula)

H22 Pb4 C4 I8 Br4 N6
H22 Pb4 C4 I2 Br10 N6
H22 Pb4 C4 Br12 N6
H22 Pb4 C4 I6 Br6 N6
H22 Pb4 C4 I10 Br2 N6
Cs1 H15 Pb4 C3 I12 N6


In [20]:
DFT_bandgaps_test

[1.6165000000000003,
 1.6435,
 1.9423,
 1.7110999999999998,
 1.5245000000000002,
 1.5330999999999997]

In [21]:
# 使用列表推导式计算对应位置元素的差值  
gaps_test = [real - DFT for real, DFT in zip(real_gaps_test, DFT_bandgaps_test)]  
  
print(gaps_test)  # 输出新列表，包含对应位置元素的差值

[0.13349999999999973, 0.49650000000000016, 0.3376999999999999, 0.11890000000000023, 0.08549999999999991, 0.03690000000000038]


In [22]:
len(gaps_test)

6

In [23]:
new_gaps = gaps+gaps_test
new_structures = structures + structures_test
print(len(new_gaps))
print(len(new_structures))

108
108


# 直接用108个数据拟合对象

In [24]:
nfeat_bond = 10
r_cutoff = 8
gaussian_centers = np.linspace(0, r_cutoff+1, nfeat_bond)
gaussian_width = 0.5
graph_converter = CrystalGraph(cutoff=r_cutoff)
model = MEGNetModel(graph_converter=graph_converter, centers=gaussian_centers, width=gaussian_width)

2023-12-12 22:08:18.688397: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-12 22:08:19.323538: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-12 22:08:19.323800: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [25]:
model.train(new_structures, new_gaps, epochs=200)

Epoch 1/200


2023-12-12 22:08:44.024524: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f0dad9cd010 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-12-12 22:08:44.024564: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): Tesla V100-SXM2-32GB, Compute Capability 7.0
2023-12-12 22:08:44.030167: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-12-12 22:08:44.071969: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
I0000 00:00:1702390124.162463   35003 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1/1 [==============================] - 28s 28s/step - loss: 0.1445
Epoch 2/200
1/1 [==============================] - 1s 1s/step - loss: 0.1429
Epoch 3/200
1/1 [==============================] - 1s 1s/step - loss: 0.1434
Epoch 4/200
1/1 [==============================] - 1s 1s/step - loss: 0.1430
Epoch 5/200
1/1 [==============================] - 1s 1s/step - loss: 0.1422
Epoch 6/200
1/1 [==============================] - 1s 1s/step - loss: 0.1419
Epoch 7/200
1/1 [==============================] - 1s 1s/step - loss: 0.1418
Epoch 8/200
1/1 [==============================] - 1s 1s/step - loss: 0.1416
Epoch 9/200
1/1 [==============================] - 1s 1s/step - loss: 0.1411
Epoch 10/200
1/1 [==============================] - 1s 1s/step - loss: 0.1404
Epoch 11/200
1/1 [==============================] - 1s 1s/step - loss: 0.1399
Epoch 12/200
1/1 [==============================] - 1s 1s/step - loss: 0.1394
Epoch 13/200
1/1 [==============================] - 1s 1s/step - loss: 0.1386
Epoch

# 提取需要拟合的326个结构对象数据

In [26]:
import os
import glob

# 指定的路径
path = '/root/home/jupyter/dft_chgnet_m3gnet/CONTCAR_opt_dft_FAPbI2BrxI1-x/'

# 使用glob模块找到所有.cif后缀的文件
cif_files = glob.glob(os.path.join(path, '*.cif'))

# 打印结果
for file in cif_files:
    print(file)

/root/home/jupyter/dft_chgnet_m3gnet/CONTCAR_opt_dft_FAPbI2BrxI1-x/structure_optimizedDFT_0.cif
/root/home/jupyter/dft_chgnet_m3gnet/CONTCAR_opt_dft_FAPbI2BrxI1-x/structure_optimizedDFT_1.cif
/root/home/jupyter/dft_chgnet_m3gnet/CONTCAR_opt_dft_FAPbI2BrxI1-x/structure_optimizedDFT_2.cif
/root/home/jupyter/dft_chgnet_m3gnet/CONTCAR_opt_dft_FAPbI2BrxI1-x/structure_optimizedDFT_3.cif
/root/home/jupyter/dft_chgnet_m3gnet/CONTCAR_opt_dft_FAPbI2BrxI1-x/structure_optimizedDFT_4.cif
/root/home/jupyter/dft_chgnet_m3gnet/CONTCAR_opt_dft_FAPbI2BrxI1-x/structure_optimizedDFT_5.cif
/root/home/jupyter/dft_chgnet_m3gnet/CONTCAR_opt_dft_FAPbI2BrxI1-x/structure_optimizedDFT_6.cif
/root/home/jupyter/dft_chgnet_m3gnet/CONTCAR_opt_dft_FAPbI2BrxI1-x/structure_optimizedDFT_7.cif
/root/home/jupyter/dft_chgnet_m3gnet/CONTCAR_opt_dft_FAPbI2BrxI1-x/structure_optimizedDFT_8.cif
/root/home/jupyter/dft_chgnet_m3gnet/CONTCAR_opt_dft_FAPbI2BrxI1-x/structure_optimizedDFT_9.cif
/root/home/jupyter/dft_chgnet_m3gnet/CON

In [27]:
len(cif_files)

326

In [29]:
predictions = []

In [30]:
for i in cif_files:
    structure = Structure.from_file(i)
    prediction = model.predict_structure(structure)
    predictions.append(prediction)

/opt/tefscloud/miniconda3/lib/python3.9/site-packages/pymatgen/io/cif.py:1219: UserWarning: Issues encountered while parsing CIF: 4 fractional coordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))
/opt/tefscloud/miniconda3/lib/python3.9/site-packages/pymatgen/io/cif.py:1219: UserWarning: Issues encountered while parsing CIF: 6 fractional coordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))
/opt/tefscloud/miniconda3/lib/python3.9/site-packages/pymatgen/io/cif.py:1219: UserWarning: Issues encountered while parsing CIF: 2 fractional coordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))
/opt/tefscloud/miniconda3/lib/python3.9/site-packages/pymatgen/io/cif.py:1219: UserWarning:

In [31]:
predictions_ = [pred[0].item() for pred in predictions]
predictions_

[0.10977491736412048,
 0.12832312285900116,
 0.1319807916879654,
 0.1108892410993576,
 0.1110457181930542,
 0.1256152093410492,
 0.1234748512506485,
 0.13283714652061462,
 0.1411285549402237,
 -0.06430667638778687,
 0.13416624069213867,
 0.11434434354305267,
 0.0852883979678154,
 0.032391052693128586,
 0.1746450960636139,
 0.16533219814300537,
 0.1819177120923996,
 0.15772046148777008,
 0.09006058424711227,
 0.16015522181987762,
 0.09602054953575134,
 0.14563310146331787,
 0.14775604009628296,
 0.07458222657442093,
 -0.10224819928407669,
 -0.004947999957948923,
 0.20099112391471863,
 0.1811460554599762,
 0.21278071403503418,
 0.18108630180358887,
 0.09314489364624023,
 0.1538996547460556,
 0.09864497184753418,
 0.08764832466840744,
 0.09513208270072937,
 0.07849404215812683,
 0.12765657901763916,
 0.19603218138217926,
 0.2091313898563385,
 0.1691998690366745,
 0.19364050030708313,
 0.18605539202690125,
 0.23767010867595673,
 0.23024079203605652,
 0.2303214818239212,
 0.2253964245319366

In [32]:
len(predictions_)

326

In [34]:
pred_structures = []
for i in cif_files:
    structure = Structure.from_file(i)
    pred_structures.append(structure)
len(pred_structures)

/opt/tefscloud/miniconda3/lib/python3.9/site-packages/pymatgen/io/cif.py:1186: UserWarning: The default value of primitive was changed from True to False in https://github.com/materialsproject/pymatgen/pull/3419. CifParser now returns the cell in the CIF file as is. If you want the primitive cell, please set primitive=True explicitly.
  warnings.warn(
/opt/tefscloud/miniconda3/lib/python3.9/site-packages/pymatgen/io/cif.py:1219: UserWarning: Issues encountered while parsing CIF: 7 fractional coordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))
/opt/tefscloud/miniconda3/lib/python3.9/site-packages/pymatgen/io/cif.py:1219: UserWarning: Issues encountered while parsing CIF: 4 fractional coordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))
/opt/tefscloud/miniconda3/lib/python3.9/

326

In [35]:
len(pred_structures)

326

# 预测对象存储

In [38]:
# 遍历测试集的结构和预测gaps值，以及提取的DFT带隙值和真实gap值 
results = []
for structure, pre in zip(pred_structures, predictions_):  
    # 创建一个字典来存储当前结构的数据  
    item = {}  
    item['structure'] = structure.as_dict()  # 将Structure对象转换为字典  
    
    item['prediction_gaps'] = pre  # 直接使用预测gaps值，因为它已经是一个浮点数了  

    results.append(item)  

print(len(results))

326


In [39]:
!pwd

/root/home/jupyter/JupyterLab/bandgap-train


In [40]:
import json  
  
# 将结果列表转换为JSON格式  
results_json = json.dumps(results)  
  
# 将JSON数据写入文件  
with open('./bandgap-fapbi2brxi1-xbrx/predicition_gap_DFT_fapbi2br.json', 'w') as file:  
    file.write(results_json)

In [41]:
!nvidia-smi

Tue Dec 12 22:40:28 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.12             Driver Version: 535.104.12   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla V100-SXM2-32GB           On  | 00000000:00:08.0 Off |                    0 |
| N/A   37C    P0              38W / 300W |  32288MiB / 32768MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [3]:
!kill -s 9  49802

In [4]:
!nvidia-smi

Tue Dec 12 22:46:02 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.12             Driver Version: 535.104.12   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla V100-SXM2-32GB           On  | 00000000:00:08.0 Off |                    0 |
| N/A   36C    P0              38W / 300W |  31474MiB / 32768MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--